In [9]:
from config import Config
import os
from dotenv import load_dotenv
load_dotenv()

openai_client = Config.get_openai_client()
openai_deployment = Config.get_openai_deployment()

agent_names = [
  "Simple Agent",
  "Spreadsheet Agent",
  "Tool Agent",
  "Googlesheet Agent",
  "Message Action Agent",
]

# inititialize all of the agents that we will be using
all_assistants = openai_client.beta.assistants.list(
    order="desc",
    limit="20",
)
existing_assistants = {assistant.name: assistant.id for assistant in all_assistants.data if assistant.name in agent_names}
existing_assistants_names = list(existing_assistants.keys())
print(f"Existing assistants: {existing_assistants_names}")

for name in agent_names:
  if name in existing_assistants_names:
    response = openai_client.beta.assistants.delete(existing_assistants[name])
    print('Deleted assistant:', name)
  new_assistant = openai_client.beta.assistants.create(
    instructions="",
    name=name,
    tools=[],
    model=openai_deployment
  )
  print('Created simple assistant:', name)

all_assistants = openai_client.beta.assistants.list(
    order="desc",
    limit="20",
)
existing_assistants = {assistant.name: assistant.id for assistant in all_assistants.data if assistant.name in agent_names}
print(f"Existing assistants: {existing_assistants}")

def ask_question (question, thread, assistant):

  openai_client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=question,
  )

  run = openai_client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
  )

  if run.status == 'completed': 
    messages = openai_client.beta.threads.messages.list(
      thread_id=thread.id
    )
    for message in reversed(messages.data):
      content = "\n".join([part.text.value for part in message.content])
      print(f"{message.role}: {content}")
  else:
    print(run.status)



Existing assistants: []
Created simple assistant: Simple Agent
Created simple assistant: Spreadsheet Agent
Created simple assistant: Tool Agent
Created simple assistant: Googlesheet Agent
Created simple assistant: Message Action Agent
Existing assistants: {'Message Action Agent': 'asst_LEm4q9F9v9lRfsAK4jei8Q3f', 'Googlesheet Agent': 'asst_JZRsPJgm8liSVEFFPF2WNcAS', 'Tool Agent': 'asst_a7RpPayDP2l2t57ZXFV3XOlj', 'Spreadsheet Agent': 'asst_Dpa8COYxmqBfNAFNZJNTI2Ml', 'Simple Agent': 'asst_b9Mqvj7uu7K6vf7kdJA1ZnLR'}


In [10]:
# create a simple agent that barely changes the default behavior

simple_assistant = openai_client.beta.assistants.update(
    assistant_id=existing_assistants["Simple Agent"],
    name="Simple Agent",
    instructions="You answer all quetions like shakespeare",
    model=openai_deployment,
)


In [11]:
# ask the simple agent a question
thread = openai_client.beta.threads.create()
ask_question("Tell me a joke", thread, simple_assistant)

user: Tell me a joke
assistant: Verily, good sir or madam, I present to thee a jest: 

Why doth the fool with spectacles make merry in the garden?

Forsooth, because he heard 'twas the best place to improve his peas of sight!


In [12]:
# ask another questions
ask_question("Tell me another one", thread, simple_assistant)

user: Tell me a joke
assistant: Verily, good sir or madam, I present to thee a jest: 

Why doth the fool with spectacles make merry in the garden?

Forsooth, because he heard 'twas the best place to improve his peas of sight!
user: Tell me another one
assistant: Hark! Anon, a jest to tickle thy fancy:

Why didst the quill cease to commune as a writing tool?

For it could no longer hold its ink-cantation, declaring itself truly unpenworthy!


In [13]:
# create an agent that uses an uploaded spreadsheet to answer questions

# first upload the file to the assistant
data_file = os.getenv("CUSTOMER_DATA_FILE")
file = openai_client.files.create(
  file=open(data_file, "rb"),
  purpose='assistants'
)

# create the agent with the uploaded file
spreadsheet_assistant = openai_client.beta.assistants.update(
    assistant_id=existing_assistants["Spreadsheet Agent"],
    name="Spreadsheet Agent",
    instructions="""
    Use the spreadsheet to answer questions about customers. The attached spreadsheet
    constains information about the customers in CSV format. The first row contains the
    names of the columns. 
    """,
    tools=[{"type": "code_interpreter"}],
    tool_resources={
      "code_interpreter": {
        "file_ids": [file.id]
      }
    },
    model=openai_deployment,
)

In [14]:
thread = openai_client.beta.threads.create()
ask_question("How many customers are there", thread, spreadsheet_assistant)


user: How many customers are there
assistant: There are 200 customers in the dataset.


In [15]:
# create an agent that uses a tool written as a python method to answer questions
# the python method 'use_numbers' lives in fumctions.py

simpletool_assistant = openai_client.beta.assistants.update(
    assistant_id=existing_assistants["Tool Agent"],
    name="Tool Agent",
    instructions="""
    You are a helpful assistant that answers questions about numbers. When the user
    gives you two numbers, you will use them to call the function 'use_numbers'. The use
    the result of that function to answer the question.
    """,
    tools=[{"type": "code_interpreter"},
       {
        "type": "function",
        "function": {
          "name": "use_numbers",
          "description": "use two numbers to generate a result",
          "parameters": {
            "type": "object",
            "properties": {
              "a": {
                "type": "integer",
                "description": "first number"
              },
              "b": {
                "type": "integer",
                "description": "second number"
              },
            },
            "required": ["a", "b"]
          }
        }
      }],
    model=openai_deployment,
)


In [16]:
# create an agent that uses a tool written as a python method to answer questions
# the python method 'execute_sql_googlesheets' lives in fumctions.py
# this function will execute a SQL query on a google sheet

googlesheet_assistant = openai_client.beta.assistants.update(
    assistant_id=existing_assistants["Googlesheet Agent"],
    name="Googlesheet Agent",
    instructions="""
    You are an expert answering questions about customers. When asked a question, you should
    translate the question into a SQL query and execute it using the function execute_sql_googlesheets.
    The table name is 'df' and the columns are:
    - Customer_ID
    - First_Name
    - Last_Name
    - Email
    - Phone_Number
    - Region
    - Salesperson
    - Address
    - ZIP_Code
    - Last_Purchase_Amount

    An example question might be "How many customers are in each regio" which you would translate into:
    SELECT Region, COUNT(*) FROM df GROUP BY Region
    """,
    tools=[{"type": "code_interpreter"},
       {
        "type": "function",
        "function": {
          "name": "execute_sql_googlesheets",
          "description": "execute a sql query on a google sheet",
          "parameters": {
            "type": "object",
            "properties": {
              "sql_query": {
                "type": "string",
                "description": "the query to execute"
              }
            },
            "required": ["sql_query"]
          }
        }
      }],
    model=openai_deployment,
)




In [ ]:
# create an agent that uses two tools written as python methods to answer requests
# both python methods lives in fumctions.py
# the function 'execute_sql_googlesheets' will execute a SQL query on a google sheet
# the function 'send_email' will send an email

message_assistant = openai_client.beta.assistants.update(
    assistant_id=existing_assistants["Message Action Agent"],
    name="Message Action Agent",
    instructions="""
    You are an expert answering questions about customers. When asked a question, you should
    translate the question into a SQL query and execute it using the function execute_sql_googlesheets.
    The table name is 'df' and the columns are:
    - Customer_ID
    - First_Name
    - Last_Name
    - Email
    - Phone_Number
    - Region
    - Salesperson
    - Address
    - ZIP_Code
    - Last_Purchase_Amount

    An example question might be "How many customers are in each regio" which you would translate into:
    SELECT Region, COUNT(*) FROM df GROUP BY Region

    You can also send a message with results for any user questions using the function 'send_email'.
    THis function takes a message body, a subject and a to email address as parameters. 
    """,
    tools=[{"type": "code_interpreter"},
       {
        "type": "function",
        "function": {
          "name": "execute_sql_googlesheets",
          "description": "execute a sql query on a google sheet",
          "parameters": {
            "type": "object",
            "properties": {
              "sql_query": {
                "type": "string",
                "description": "the query to execute"
              }
            },
            "required": ["sql_query"]
          }
        }
      },
      {
        "type": "function",
        "function": {
          "name": "send_email",
          "description": "send an email",
          "parameters": {
            "type": "object",
            "properties": {
              "to": {
                "type": "string",
                "description": "the to address"
              },
              "subject": {
                "type": "string",
                "description": "the subject of the email"
              },
              "body": {
                "type": "string",
                "description": "the body of the email"
              }
            },
            "required": ["to", "subject", "body"]
          }
        }
      },
      ],
    model=openai_deployment,
)